In [ ]:
let readHex = 
  Seq.collect (function
  | '0' -> [0;0;0;0]
  | '1' -> [0;0;0;1]
  | '2' -> [0;0;1;0]
  | '3' -> [0;0;1;1]
  | '4' -> [0;1;0;0]
  | '5' -> [0;1;0;1]
  | '6' -> [0;1;1;0]
  | '7' -> [0;1;1;1]
  | '8' -> [1;0;0;0]
  | '9' -> [1;0;0;1]
  | 'A' -> [1;0;1;0]
  | 'B' -> [1;0;1;1]
  | 'C' -> [1;1;0;0]
  | 'D' -> [1;1;0;1]
  | 'E' -> [1;1;1;0]
  | 'F' -> [1;1;1;1]
  )
  >> Seq.toList

let input = (File.ReadAllText "input.txt").Trim() |> readHex

let binToInt = List.fold (fun v bit -> v * 2 + bit) 0
let binToLong = List.fold (fun v bit -> v * 2L + int64 bit) 0L

let (|ReadBits|_|) n lst = 
  if List.length lst < n then None else
  let (bits, rest) = List.splitAt n lst
  Some (binToInt bits, rest)

let (|PacketHeader|_|) = function
  | ReadBits 3 (version, ReadBits 3 (typeId, rest)) -> Some (version, typeId, rest)
  | _ -> None

let rec readLiteralContents = function
  | ReadBits 1 (1, rst) -> 
    let (bits, rest) = List.splitAt 4 rst
    let (otherBits, finalRest) = readLiteralContents rest
    (bits @ otherBits, finalRest)
  | ReadBits 1 (0, rst) -> List.splitAt 4 rst

type Op = 
  | Sum
  | Product
  | Min
  | Max
  | GreaterThan
  | LessThan
  | Equal

let (|Op|) = function
  | 0 -> Sum
  | 1 -> Product
  | 2 -> Min
  | 3 -> Max
  | 5 -> GreaterThan
  | 6 -> LessThan
  | 7 -> Equal

type Packet = 
  | Literal of int * int64
  | Operator of int * Op * Packet list

let (|LiteralPacket|_|) = function
  | PacketHeader (version, 4, rst) -> 
    let (bits, rest) = readLiteralContents rst
    Some (Literal (version, binToLong bits), rest)
  | _ -> None

let rec (|OperatorPacket|_|) = function
  | PacketHeader (v, Op t, ReadBits 1 (0, ReadBits 15 (len, rst))) ->
    let (inner, rest) = rst |> List.splitAt len
    Some (Operator (v, t, parse inner), rest) 
  | PacketHeader (v, Op t, ReadBits 1 (1, ReadBits 11 (packetCount, rst))) ->
    let (packets, rest) = 
      [1..packetCount]
      |> List.fold (fun (pkts, lst) _ -> 
        match parse1 lst with
        | Some (v, r) -> (v :: pkts, r)
        | None -> failwith "Ran out of bits") ([], rst)
    Some (Operator (v, t, packets |> List.rev), rest)
  | _ -> None
and parse1 = function
  | [] -> None
  | LiteralPacket (value, rest) -> Some (value, rest)
  | OperatorPacket (value, rest) -> Some (value, rest)
  | lst when List.forall ((=) 0) lst -> None
  | e -> failwith (sprintf "%A" e)
and parse lst =
  match parse1 lst with
  | Some (value, rest) -> value :: parse rest
  | None -> []

let rec countVersion = function
  | Literal (ver, _) -> ver
  | Operator (ver, _, pkts) -> ver + Seq.sumBy countVersion pkts

parse input
|> Seq.sumBy countVersion


965

In [ ]:
let exec = function
  | Sum -> Seq.sum
  | Product -> Seq.reduce (*)
  | Min -> Seq.min
  | Max -> Seq.max
  | GreaterThan -> fun [x; y] -> if x > y then 1L else 0L
  | LessThan -> fun [x; y] -> if x < y then 1L else 0L
  | Equal -> fun [x; y] -> if x = y then 1L else 0L
  | _ -> failwith "Invalid op"

let rec eval = function
  | Literal (_, v) -> v
  | Operator (_, op, pkts) ->
    pkts 
    |> List.map eval
    |> exec op

let evalHex str =
  str
  |> readHex
  |> parse
  |> Seq.exactlyOne
  |> eval

parse input
|> Seq.exactlyOne
|> eval

116672213160